# AutoGen Temel Örnek

Bu kod örneğinde, [AutoGen](https://aka.ms/ai-agents/autogen) Yapay Zeka Çerçevesini kullanarak temel bir ajan oluşturacaksınız.

Bu örneğin amacı, farklı ajan desenlerini uygularken ilerideki ek kod örneklerinde kullanacağımız adımları size göstermektir.


## Gerekli Python Paketlerini İçe Aktarın


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## İstemciyi Oluştur

Bu örnekte, LLM'e erişim için [GitHub Modelleri](https://aka.ms/ai-agents-beginners/github-models) kullanacağız.

`model`, `gpt-4o-mini` olarak tanımlanmıştır. Farklı sonuçları görmek için modeli GitHub Modelleri pazarında bulunan başka bir modelle değiştirmeyi deneyin.

Hızlı bir test olarak, sadece basit bir istem çalıştıracağız - `Fransa'nın başkenti nedir`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Ajanı Tanımlama

`client`i kurup çalıştığını doğruladığımıza göre, şimdi bir `AssistantAgent` oluşturalım. Her bir ajana şu özellikler atanabilir:  
**name** - Çoklu ajan akışlarında referans vermek için kullanışlı olan kısa bir isim.  
**model_client** - Önceki adımda oluşturduğunuz istemci.  
**tools** - Ajanın bir görevi tamamlamak için kullanabileceği mevcut araçlar.  
**system_message** - LLM'nin görevi, davranışı ve tonunu tanımlayan metaprompt.  

LLM'nin nasıl tepki verdiğini görmek için system message'ı değiştirebilirsiniz. `tools` konusunu 4. Derste ele alacağız.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Ajanı Çalıştır

Aşağıdaki fonksiyon ajanı çalıştıracaktır. Ajanın durumunu yeni mesajla güncellemek için `on_message` metodunu kullanıyoruz.

Bu durumda, kullanıcıdan gelen `"Bana harika bir güneşli tatil planla"` mesajıyla durumu güncelliyoruz.

Mesaj içeriğini değiştirerek LLM'nin farklı şekilde nasıl yanıt verdiğini görebilirsiniz.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Feragatname**:  
Bu belge, [Co-op Translator](https://github.com/Azure/co-op-translator) adlı yapay zeka çeviri hizmeti kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlıklar içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlama veya yanlış yorumlamalardan sorumlu değiliz.
